In [16]:
import pandas as pd
import numpy as np
import yfinance as yf
from scipy.optimize import minimize

In [ ]:
# tickers utilizados
tickers = {
    "BVSP": "^BVSP",
    "SP500": "^GSPC",
    "USDBRL": "USDBRL=X",
    "GOLD": "GC=F"
} 

data = yf.download(list(tickers.values()), start="2020-01-02", end="2024-12-30")['Close'] # data pedida
data.columns = tickers.keys()

# log-retornos 
returns = np.log(data/data.shift(1)).dropna()

# usd --> brl, considerando log
returns['SP500'] = returns['SP500'] + returns['USDBRL']
returns['GOLD']  = returns['GOLD']  + returns['USDBRL']


In [17]:
# inicio do black-litterman
assets = ["BVSP", "SP500", "USDBRL", "GOLD"]
R = returns[assets]
Sigma = R.cov().values
n = len(assets)

# parâmetros 
delta = 2.5 # chutes iniciais, mas uma estimativa melhor seria com base no retorno do mercado e no ret. livre de risco
tau = 0.025 # o mesmo vale aqui

w_eq = np.ones(n)/n
Pi = delta * Sigma.dot(w_eq)


In [18]:
# visao P: Ouro - S&P500 = +2%
P = np.array([[0, -1, 0, 1]])
Q = np.array([0.02])
Omega = np.diag(np.diag(P @ (tau * Sigma) @ P.T))


In [19]:
M_inv = np.linalg.inv(np.linalg.inv(tau * Sigma) + P.T @ np.linalg.inv(Omega) @ P)
mu_bl = M_inv @ (np.linalg.inv(tau * Sigma) @ Pi + P.T @ np.linalg.inv(Omega) @ Q)


In [ ]:
# objetivo a ser minimizado 
def objective(w, mu, Sigma, delta):
    return -(w @ mu - 0.5 * delta * w @ Sigma @ w)

constraints = {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}
bounds = [(0, 1)] * n
w0 = np.ones(n)/n

res = minimize(objective, w0, args=(mu_bl, Sigma, delta), # minimização
               method='SLSQP', bounds=bounds, constraints=constraints)
w_bl_long = res.x
allocation = pd.Series(w_bl_long, index=assets)


In [ ]:
# retorno diário 
portfolio_returns = (R * w_bl_long).sum(axis=1)
portfolio_value = (1 + portfolio_returns).cumprod() * 100000

# retorno e risco anual
def annual_stats(series):
    return series.resample('Y').agg(['sum','std'])

annual_portfolio = annual_stats(portfolio_returns)
annual_assets = {a: annual_stats(R[a]) for a in assets}

# inflação (em %)
ipca = {
    2020: 4.52, 2021: 10.06, 2022: 5.79, 2023: 4.62, 2024: 4.83
}

# VIX anual, usando yfinance
vix = yf.download('^VIX', start="2020-01-02", end="2024-12-30")['Close']
vix_annual = vix.resample('Y').mean()


C:\Users\arthu\AppData\Local\Temp\ipykernel_75832\1634207949.py:7: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  return series.resample('Y').agg(['sum','std'])
C:\Users\arthu\AppData\Local\Temp\ipykernel_75832\1634207949.py:18: FutureWarning: YF.download() has changed argument auto_adjust default to True
  vix = yf.download('^VIX', start="2020-01-02", end="2024-12-30")['Close']
[*********************100%***********************]  1 of 1 completed
C:\Users\arthu\AppData\Local\Temp\ipykernel_75832\1634207949.py:19: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  vix_annual = vix.resample('Y').mean()


In [21]:
years = annual_portfolio.index.year

df_summary = pd.DataFrame({
    'Retorno Portfólio (%)': (annual_portfolio['sum'] * 100).values,
    'Risco Portfólio (%)': (annual_portfolio['std'] * np.sqrt(252) * 100).values,
    'VIX (%)': vix_annual.values.squeeze(), 
    'Inflação (%)': years.map(ipca).values
}, index=years)

In [22]:
df_summary

,Retorno Portfólio (%),Risco Portfólio (%),VIX (%),Inflação (%)
Date,,,,
2020,23.299518,77.874853,29.251304,4.52
2021,10.058518,29.076621,19.656389,10.06
2022,2.718181,38.151436,25.623904,5.79
2023,50.592135,24.601239,16.870040,4.62
2024,19.155552,20.367826,15.593440,4.80
